In [36]:
import os
import pickle
import numpy as np
from sparse_to_array import *
from cluster_metrics import *
from scipy import spatial
from scipy import pi
import numpy as np
from sklearn import cluster
import text_clustering
from heapq import *
import pandas as pd
from importlib import reload
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re
import time
from operator import itemgetter
#reload(sparse_to_array)
reload(text_clustering)
%matplotlib inline

In [37]:
os.chdir('/home/matt/Git/TextMiningFinal/')

In [38]:
vector_dir = 'data/vectors/'
topic_dir = 'data/topics/'
results_dir = 'results/'
#os.chdir('..')
#os.chdir(datadir)

In [39]:
tfidf_corpora = os.listdir(vector_dir)
topic_corpora = os.listdir(topic_dir)

In [40]:
# keep only the doc-topic weights
topic_corpora = [name for name in topic_corpora if re.search('COMPOSITION',name)]

In [41]:
topic_corpora

['20_newsgroups_simple.mallet.5.topics_COMPOSITION.txt',
 'reuters_simple.mallet.10.topics_COMPOSITION.txt',
 '20_newsgroups_simple.mallet.20.topics_COMPOSITION.txt',
 '20_newsgroups_complex.mallet.10.topics_COMPOSITION.txt',
 '20_newsgroups_complex.mallet.40.topics_COMPOSITION.txt',
 '20_newsgroups_complex.mallet.80.topics_COMPOSITION.txt',
 '20_newsgroups_simple.mallet.80.topics_COMPOSITION.txt',
 '20_newsgroups_simple.mallet.10.topics_COMPOSITION.txt',
 '20_newsgroups_simple.mallet.160.topics_COMPOSITION.txt',
 '20_newsgroups_complex.mallet.160.topics_COMPOSITION.txt',
 'reuters_simple.mallet.5.topics_COMPOSITION.txt',
 '20_newsgroups_simple.mallet.40.topics_COMPOSITION.txt',
 '20_newsgroups_complex.mallet.20.topics_COMPOSITION.txt',
 '20_newsgroups_complex.mallet.5.topics_COMPOSITION.txt',
 'reuters_simple.mallet.160.topics_COMPOSITION.txt',
 'reuters_simple.mallet.20.topics_COMPOSITION.txt',
 'reuters_simple.mallet.40.topics_COMPOSITION.txt',
 'reuters_simple.mallet.80.topics_COMP

In [42]:
tfidf_corpora

['20_newsgroups_complex.p', '20_newsgroups_simple.p', 'reuters_simple.p']

# Get the document labels

In [43]:
with open(os.path.join(vector_dir,'reuters_simple.p'),'rb') as infile:
    labels = pickle.load(infile)
labels = [(doc[0],doc[1]) for doc in labels[1]]
labels.sort(key=itemgetter(0))
reuters_labels = [itemgetter(1)(tup) for tup in labels]
print(reuters_labels[0:10])

['money-supply', 'trade', 'crude', 'earn', 'interest', 'trade', 'strategic-metal', 'trade', 'earn', 'earn']


In [44]:
with open(os.path.join(vector_dir,'20_newsgroups_simple.p'),'rb') as infile:
    labels = pickle.load(infile)
labels = [(doc[0],doc[1]) for doc in labels[1]]
labels.sort(key=itemgetter(0))
newsgroups_labels = [itemgetter(1)(tup) for tup in labels]
print(newsgroups_labels[0:10])

[7, 4, 4, 1, 14, 16, 13, 3, 2, 4]


In [45]:
newsgroups_labels = np.array(newsgroups_labels)

# turn reuters string labels into ints
reuters_labels = np.array(reuters_labels)
enc = LabelEncoder()
reuters_labels = enc.fit_transform(reuters_labels)

In [46]:
# both sets of labels should be numpy arrays
print(type(reuters_labels))
print(type(newsgroups_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Set up a dataframe to store the validation results

In [27]:
def clean_corpus_name(name):
    return os.path.splitext(name)[0].strip("_COMPOSITION")

In [28]:
corpus_names = [clean_corpus_name(name) for name in topic_corpora+tfidf_corpora]
results_index = pd.Index(corpus_names)

# the metrics we'll use
node_metrics = {'label_averaged_max_F1': lambda labels,clust: label_averaged_score(labels,clust,metrics.f1_score),
                'cluster_averaged_max_F1': lambda labels,clust: cluster_averaged_score(labels,clust,metrics.f1_score)}
level_metrics = {'NMI_by_max': NMI_by_max, 'NMI_by_mean': NMI_by_mean, 
                 'ARI': lambda labels,cluster_labels,label_h: metrics.adjusted_rand_score(labels,cluster_labels)}

results = pd.DataFrame(index=results_index, columns=list(level_metrics.keys())+list(node_metrics.keys())+['runtime'])

In [29]:
results

,NMI_by_mean,NMI_by_max,ARI,cluster_averaged_max_F1,label_averaged_max_F1,runtime
20_newsgroups_simple.mallet.5.topics,NaN,NaN,NaN,NaN,NaN,NaN
reuters_simple.mallet.10.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_simple.mallet.20.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_complex.mallet.10.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_complex.mallet.40.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_complex.mallet.80.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_simple.mallet.80.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_simple.mallet.10.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_simple.mallet.160.topics,NaN,NaN,NaN,NaN,NaN,NaN
20_newsgroups_complex.mallet.160.topics,NaN,NaN,NaN,NaN,NaN,NaN


# Perform validations on all datasets

In [32]:
reload(text_clustering)
for datafile in topic_corpora:
    name = clean_corpus_name(datafile)
    topicArray = topic_composition_to_array(os.path.join(topic_dir,datafile))
    
    if re.search('reuters',datafile):
        labels = reuters_labels#[0:1000]
        label_H = label_entropy(reuters_labels)
    if re.search('newsgroups',datafile):
        labels = newsgroups_labels#[0:1000]
        label_H = label_entropy(newsgroups_labels)
    
    topicArray = np.sqrt(topicArray)
    topicArray = topicArray#[0:1000,:]
    
    clust = text_clustering.BisectingPartitional(candidate_splits=10,objective=text_clustering.I2,max_leaves=200)
    
    print("Clustering {}".format(name))
    print("shape: {}".format(topicArray.shape))
    print()
    start = time.time()
    clust.fit(topicArray)
    end = time.time()
    
    results.loc[name,'runtime'] = end - start
    print("runtime: {}".format(end-start))
    
    score_dict = {}
    for metric_name, metric in level_metrics.items():
        score_dict[metric_name] = np.zeros(clust.num_leaves)
    score_dict['objective'] = np.zeros(clust.num_leaves)
    
    for i,clusters in enumerate(clust.levels(n=clust.num_leaves,return_labels=True)):
        #print(clusters[1])
        #print(i)
        score_dict['objective'][i] = clusters[1]
        for metric_name, metric in level_metrics.items():
            score_dict[metric_name][i] = metric(labels,clusters[0],label_H)
        
    for metric_name, scores in score_dict.items():
        best_score = max(scores)
        print("{}: {}".format(metric_name,best_score))
        results.loc[name,metric_name] = best_score

    #score_dict[metric_name] = scores
    #best_score = max(scores)
    #results.loc[name,metric_name] = best_score

    with open(os.path.join(results_dir,name+'.p'),'wb') as outfile:
        pickle.dump(score_dict,outfile)

    for metric_name, metric in node_metrics.items():
        score = metric(labels,clust)
        print("{}: {}".format(metric_name,score))
        results.loc[name,metric_name] = score
        
    # save the intermediate topic clustering results
    results.to_csv(os.path.join(results_dir,'results.csv'))
    
    print()

shape: (11314, 5)
Clustering 20_newsgroups_simple.mallet.5.topics

runtime: 2.127028226852417
NMI_by_mean: 0.45837625264316295
NMI_by_max: 0.42503407063756915
objective: 525.6051780481865
ARI: 0.203189352499786
cluster_averaged_max_F1: 0.24995759078558571
label_averaged_max_F1: 0.3421483121560402

shape: (9086, 10)
Clustering reuters_simple.mallet.10.topics

runtime: 2.10750412940979
NMI_by_mean: 0.5745930421677299
NMI_by_max: 0.5581128379037306
objective: 713.5805927065005
ARI: 0.48301529662699183
cluster_averaged_max_F1: 0.41449254520518536
label_averaged_max_F1: 0.664646432297632

shape: (11314, 20)
Clustering 20_newsgroups_simple.mallet.20.topics

runtime: 2.4505040645599365
NMI_by_mean: 0.5680132770449237
NMI_by_max: 0.5440527449833417
objective: 644.9537855771603
ARI: 0.3413360503870071
cluster_averaged_max_F1: 0.291063024617013
label_averaged_max_F1: 0.5160994453987419

shape: (11314, 10)
Clustering 20_newsgroups_complex.mallet.10.topics

runtime: 2.281630516052246
NMI_by_mean: 

/home/matt/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [ ]:
# save the intermediate topic clustering results
with open(os.path.join(results_dir,'results.p'),'wb') as outfile:
    pickle.dump(results,outfile)
results.to_csv(os.path.join(results_dir,'results.csv'))

In [35]:
results

,NMI_by_mean,NMI_by_max,ARI,cluster_averaged_max_F1,label_averaged_max_F1,runtime,objective
20_newsgroups_simple.mallet.5.topics,0.458376,0.425034,0.203189,0.249958,0.342148,2.12703,525.605178
reuters_simple.mallet.10.topics,0.574593,0.558113,0.483015,0.414493,0.664646,2.1075,713.580593
20_newsgroups_simple.mallet.20.topics,0.568013,0.544053,0.341336,0.291063,0.516099,2.4505,644.953786
20_newsgroups_complex.mallet.10.topics,0.486552,0.428572,0.222852,0.268321,0.388413,2.28163,509.215176
20_newsgroups_complex.mallet.40.topics,0.56315,0.459362,0.241095,0.246904,0.491256,2.30289,476.727558
20_newsgroups_complex.mallet.80.topics,0.521093,0.413776,0.154448,0.226828,0.399014,2.68211,502.432354
20_newsgroups_simple.mallet.80.topics,0.571593,0.490729,0.263647,0.262163,0.556534,3.06716,665.304520
20_newsgroups_simple.mallet.10.topics,0.516167,0.505222,0.291224,0.285585,0.438305,2.43256,622.317789
20_newsgroups_simple.mallet.160.topics,0.514346,0.410761,0.144834,0.195817,0.400035,3.8446,691.023438
20_newsgroups_complex.mallet.160.topics,0.466429,0.36993,0.099461,0.196189,0.332588,3.03819,533.116826
